## Extract the level data from the source files

In [240]:
"""
Pull out all the level data from the source files.
"""
files = ["../iridisalpha/src/level_data/level_data.asm",
         "../iridisalpha/src/level_data/level_data2.asm"]
d = {}
level_data = {}
ch = None
ck = None
for f in files:
    ld = open(f, 'r')
    while True:
        l = ld.readline()
        if not l:
            break
        if "pieceOfPlanet" in l:
            continue
        if l[:2].strip():
            if ch and d:
                level_data[ch] = d
            ch = l.split()[0].strip()
            d = {}
            continue

        k = l[10:17]
        if "Byte" in k:
            ck = k
            continue

        if "BYTE" in l:
            v = l[14:].strip().split(',')[0].replace('<','')
            d[ck] = v

# Drop any spurious entries
level_data = {k:c for k,c in level_data.items() if len(c) == 30}
len(level_data.keys())

200

## Look at unused data

In [235]:
# Get the settings which are never used.

def createSimpleTable(cands):
    cols = list(list(cands.values())[0].keys())
    tb = []
    for p,d in level_data.items():
        tb += [list(d.values())]
    return (tb, cols)

level_table,cols = createSimpleTable(level_data)
level_fields = [l for l in level_table]
for i in range(0,max([len(l) for l in level_fields])):
        m = set([l[i] for l in level_fields])
        if len(m) > 2:
            continue
        print(cols[i], sorted(m))

Byte 11 ['nullPtr']
Byte 12 ['$00']
Byte 14 ['nullPtr']
Byte 15 ['$00', '$01']
Byte 34 ['$00', '$14']


In [238]:
l = ["Byte 15", "Byte 34"]
for k,c in level_data.items():
    for i in l:
        if c[i] != "$00":
            print(i, k, c[i])

Byte 34 planet5Level12Data $14
Byte 15 defaultExplosion $01


## Create tables for main wave data

In [221]:
import pandas as pd
from tabulate import tabulate
from texttable import Texttable
import latextable


styles = [dict(selector="caption", 
    props=[("text-align", "center"),
    ("font-size", "120%"),
    ("color", 'black')])]

def createTable(indices, cands):
    """ Make a table and column list of the wave data"""

    tb = []
    cols = ['Planet', 'Level'] 
    all_cols = list(list(cands.values())[0].keys())
    for i in indices:
        cols += [all_cols[i]]
    for p,d in cands.items():
        t = p.replace('planet','').replace('Level', ' ').replace('Data', '')
        f = [*t.split()]
        l = list(d.values())
        for i in indices:
            if i >= len(l):
                continue
            f += [l[i]]
        tb += [f]
    level_table = sorted(tb, key=lambda x: (x[0],int(x[1])))
    return (level_table, cols)


In [216]:
# Restrict `cands` to just the guys that are the initial wave data for each
# wave. We'll look at the others separately.
cands = {c:v for c,v in sorted(level_data.items())
         if c.startswith("planet") and c.endswith("Data")}

#all_fields = list(range(0,30))
all_fields = [0,2,4,7,9,10,12,13,14,15]
level_table, cols = createTable(all_fields, cands)

# Get the unique values used in the initial level data.
level_fields = [l[2:] for l in level_table]
for i in range(0,max([len(l) for l in level_fields])):
        m = set([l[i] for l in level_fields])
        print(cols[2:][i], sorted(m))

Byte 1  ['$00', '$02', '$03', '$04', '$05', '$06', '$07', '$08', '$09', '$0A', '$0B', '$0C', '$0D', '$0E', '$0F', '$10', '$11', '$55']
Byte 3  ['$FC', 'ATARI_ST', 'BALLOON', 'BIRD1', 'BOLAS1', 'BOUNCY_RING', 'BUBBLE', 'BULLHEAD', 'CAMEL', 'COPTIC_CROSS', 'CUMMING_COCK1', 'EYE_OF_HORUS', 'FLAG_BAR', 'FLYING_COCK1', 'FLYING_COCK_RIGHT1', 'FLYING_COMMA1', 'FLYING_DART1', 'FLYING_DOT1', 'FLYING_FLOWCHART1', 'FLYING_SAUCER1', 'FLYING_TRIANGLE1', 'GILBY_AIRBORNE_LEFT', 'GILBY_AIRBORNE_RIGHT', 'LAND_GILBY1', 'LICKER_SHIP1', 'LITTLE_DART', 'LITTLE_OTHER_EYEBALL', 'LLAMA', 'LOZENGE', 'MAGIC_MUSHROOM', 'PSI', 'QBERT_SQUARES', 'SMALLBALL_AGAIN', 'SMALL_BALL1', 'SPINNING_RING1', 'STARSHIP', 'STRANGE_SYMBOL', 'TEARDROP_EXPLOSION1', 'WINGBALL']
Byte 4  ['$00', '$01', '$02', '$03', '$04', '$05', '$06', '$07']
Byte 6  ['$FC', 'ATARI_ST', 'BIRD1', 'BOLAS1', 'BOUNCY_RING', 'BUBBLE', 'BULLHEAD', 'CAMEL', 'COPTIC_CROSS', 'EYE_OF_HORUS', 'FLAG_BAR', 'FLYING_COCK1', 'FLYING_COCK_RIGHT1', 'FLYING_COMMA1', 'F

In [195]:
#all_fields = list(range(0,30))
all_fields = [0,2,4,7,9,10,12,13,14,15]
level_table, cols = createTable(all_fields)


[['1',
  '1',
  '$06',
  '$03',
  '$00',
  '$00',
  '$00',
  '$40',
  '$06',
  '$01',
  '$01',
  '$01'],
 ['1',
  '2',
  '$06',
  '$01',
  '$00',
  '$00',
  '$00',
  '$00',
  '$00',
  '$24',
  '$02',
  '$01'],
 ['1',
  '3',
  '$05',
  '$04',
  '$00',
  '$00',
  '$00',
  '$30',
  '$FA',
  '$01',
  '$01',
  '$02'],
 ['1',
  '4',
  '$11',
  '$03',
  '$00',
  '$00',
  '$00',
  '$60',
  '$07',
  '$00',
  '$01',
  '$02'],
 ['1',
  '5',
  '$11',
  '$00',
  '$00',
  '$00',
  '$00',
  '$00',
  '$FC',
  '$23',
  '$02',
  '$02'],
 ['1',
  '6',
  '$0A',
  '$03',
  '$00',
  '$00',
  '$00',
  '$03',
  '$00',
  '$00',
  '$01',
  '$01'],
 ['1',
  '7',
  '$09',
  '$00',
  '$00',
  '$00',
  '$00',
  '$50',
  '$07',
  '$00',
  '$01',
  '$02'],
 ['1',
  '8',
  '$11',
  '$03',
  '$00',
  '$00',
  '$00',
  '$00',
  '$05',
  '$00',
  '$01',
  '$02'],
 ['1',
  '9',
  '$06',
  '$03',
  '$00',
  '$00',
  '$00',
  '$0C',
  '$FC',
  '$23',
  '$01',
  '$03'],
 ['1',
  '10',
  '$08',
  '$00',
  '$00',
  '$00',
  '$

In [ ]:
df = pd.DataFrame(level_table, columns=cols)
#df = df.set_axis(sorted(cands.keys()), axis='index')

df = df.style.set_caption("Level Data in Iridis Alpha").set_table_styles(styles)
display(df)


In [192]:

rows = level_table
#print('\nTabulate Latex:')
print(tabulate(rows, headers=cols, tablefmt='latex'))

#print('\nTexttable Latex:')
#print(latextable.draw_latex(table, caption="A comparison of rocket features."))

\begin{tabular}{rrllllllllll}
\hline
   Planet &   Level & Byte 1    & Byte 4    & Byte 7    & Byte 12   & Byte 15   & Byte 16   & Byte 19   & Byte 20   & Byte 21   & Byte 22   \\
\hline
        1 &       1 & \$06       & \$03       & \$00       & \$00       & \$00       & \$40       & \$06       & \$01       & \$01       & \$01       \\
        1 &       2 & \$06       & \$01       & \$00       & \$00       & \$00       & \$00       & \$00       & \$24       & \$02       & \$01       \\
        1 &       3 & \$05       & \$04       & \$00       & \$00       & \$00       & \$30       & \$FA       & \$01       & \$01       & \$02       \\
        1 &       4 & \$11       & \$03       & \$00       & \$00       & \$00       & \$60       & \$07       & \$00       & \$01       & \$02       \\
        1 &       5 & \$11       & \$00       & \$00       & \$00       & \$00       & \$00       & \$FC       & \$23       & \$02       & \$02       \\
        1 &       6 & \$0A       & \$03       & 